# Real-word Spell Correction

### Imports

In [1]:
import regex as re
import string
import numpy as np
from collections import Counter
from nltk.util import ngrams

### Edit Distance Approach (Add non-dp approach)

In [2]:
def editDistance(str1, str2, m, n): 
    # Create a table to store results of subproblems 
    dp = [[0 for x in range(n+1)] for x in range(m+1)] 
  
    # Fill d[][] in bottom up manner 
    for i in range(m+1): 
        for j in range(n+1): 
  
            # If first string is empty, only option is to 
            # insert all characters of second string 
            if i == 0: 
                dp[i][j] = j    # Min. operations = j 
  
            # If second string is empty, only option is to 
            # remove all characters of second string 
            elif j == 0: 
                dp[i][j] = i    # Min. operations = i 
  
            # If last characters are same, ignore last char 
            # and recur for remaining string 
            elif str1[i-1] == str2[j-1]: 
                dp[i][j] = dp[i-1][j-1] 
  
            # If last character are different, consider all 
            # possibilities and find minimum 
            else: 
                dp[i][j] =     min(dp[i][j-1] + 1,        # Insert 
                                   dp[i-1][j] + 1,        # Remove 
                                   dp[i-1][j-1] + 1)    # Replace 
  
    return dp[m][n] 

In [3]:
print('Edit Distance between "form" and "from" = ',editDistance('frtm','from',4,4))

Edit Distance between "form" and "from" =  1


In [103]:
def ind(char): # Returns the index of character in alphabets
    return string.ascii_lowercase.index(char)

def get_weights():
    insert = delete = sub = trans = [[0 for i in range(26)] for j in range(26)]
    for i in open('errors.txt').readlines():
        exp,val = i.split('\t')
        val = [i for i in val.split() if i.isdigit()]
        val = int(''.join(val))
        l,r = exp.split('|')
        chars = string.ascii_lowercase
        if(len(l)==2 and len(r)==1 and l[0]==r and l[0] in chars and l[1] in chars):
            delete[ind(l[0])][ind(l[1])] = val
        elif(len(l)==1 and len(r)==2 and l==r[0] and r[0] in chars and r[1] in chars):
            insert[ind(r[0])][ind(r[1])] = val
        elif(len(l)==1 and len(r)==1 and l in chars and r in chars):
            sub[ind(l)][ind(r)] = val
        elif(len(l)==2 and len(r)==2 and l[0]==r[1] and l[1]==r[0] and l[0] in chars and l[1] in chars):
            trans[ind(l[0])][ind(l[1])] = val
    return [delete,insert,sub,trans]

def weightedEditDistance(s1,s2,m,n,insert,delete,sub):
    D = [[0 for x in range(n+1)] for y in range(m+1)]
            
    for i in range(m+1):
        for j in range(n+1):
            
            if(i==0):
                D[i][j] = j
                
            elif(j==0):
                D[i][j] = i
                
            elif(s1[i-1] == s2[j-1]):
                D[i][j] = D[i-1][j-1]
                
            else:
                a = D[i-1][j] + delete[ind(s1[i-1])]
                b = D[i][j-1] + insert[ind(s2[j-1])]
                c = D[i-1][j-1] + sub[ind(s1[i-1])][ind(s2[j-1])]
                
                D[i][j] = min(a,b,c)
    
    return D[m][n]

In [104]:
# Random errors for insertion, deletion and substitution. Can be taken from corpus with spelling mistakes and their corrections.
insert = np.random.randint(1,10,size=26)
delete = np.random.randint(1,10,size=26)
sub = np.random.randint(1,10,size=(26,26))

weightedEditDistance('from','frtm',4,4,insert,delete,sub)

4

## Building a spell Corrector

### Loading 'big.txt' - Concatenation of public domain book excerpts from Project Gutenberg

In [6]:
data = open('big.txt',encoding='utf-8').read()
print("Number of lines = ",len(data.split('\n')))
data = data.replace('\n',' ').replace('\t',' ').replace('-',' ').replace('—',' ')
data = data.replace('_','')
data = re.sub(r'[^\w\s]','',data)
data = data.lower().strip().split(' ')
data = list(filter(None,data))
print("Number of Tokens = ",len(data))

WORDS = Counter(data)

Number of lines =  128458
Number of Tokens =  1105736


### Finding closest word

In [7]:
def P(word, N=sum(WORDS.values())): 
    # Returns probability of a word in vocabulary.
    return WORDS[word] / N


def word_candidates(word):
    # Returns the words that are edit distance 0,1 or 2 away from the given word. Inspired from Peter Norvig
    ed_0 = set()
    ed_1 = set()
    ed_2 = set()
    for w in WORDS:
        ed = editDistance(word,w,len(word),len(w))
        if(ed > 2):
            continue
        elif(ed==0):
            ed_0.add(w)
        elif(ed==1):
            ed_1.add(w)
        elif(ed==2):
            ed_2.add(w)
    return [ed_0,ed_1,ed_2,{word}]

def closest_word(word):
    # Chooses closest word according to their frequency. Highest priority given to words with least edit distance.
    for i in word_candidates(word):
        if(i):
            return max(i,key=P)

In [ ]:
def P(word, N=sum(WORDS.values())): 
    # Returns probability of a word in vocabulary.
    return WORDS[word] / N


def word_candidates(word):
    # Returns the words that are edit distance 0,1 or 2 away from the given word. Inspired from Peter Norvig
    ed_0 = set()
    ed_1 = set()
    ed_2 = set()
    for w in WORDS:
        ed = editDistance(word,w,len(word),len(w))
        if(ed > 2):
            continue
        elif(ed==0):
            ed_0.add(w)
        elif(ed==1):
            ed_1.add(w)
        elif(ed==2):
            ed_2.add(w)
    return [ed_0,ed_1,ed_2,{word}]

def closest_word(word):
    # Chooses closest word according to their frequency. Highest priority given to words with least edit distance.
    for i in word_candidates(word):
        if(i):
            return max(i,key=P)

## Building a sentence corrector

### Build without assumption

### Assumption: Only 1 mistake per sentence.

In [101]:
bigrams = Counter(ngrams(data,2))
def bigram_prob(sent):
    out = P(sent[0])
    for i in range(1,len(sent)):
        out *= bigrams[(sent[i-1],sent[i])] / WORDS[sent[i-1]]
    return out

def sent_candidates(sent):
    cands = []
    sent = sent.split()
    word_present = [s in WORDS for s in sent]
    if all(word_present): 
        cands.append(sent)
        for i in range(len(sent)):
            words = word_candidates(sent[i])[1]
            for j in words:
                l = sent.copy()
                l[i] = j
                cands.append(l)
    else: 
        idx = word_present.index(0)
        words = word_candidates(sent[idx])[1]
        for i in words:
            l = sent.copy()
            l[idx] = i
            cands.append(l)
    return cands

def closest_sent(sent):
    return ' '.join(max(sent_candidates(sent),key=bigram_prob))

In [102]:
closest_sent('i have an apply')

'i have an apple'

## 25-40 % of spelling errors are real words kukich 1992

## Bigram context based spell check (with incorrect spelling)

## Trigram and quadgram context based spell check (with incorrect spelling)

## Bigram context based spell check (with correct spelling - word exists in dictionary)

## Use word embeddings